In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit

%matplotlib inline

In [2]:
from __future__ import unicode_literals
from __future__ import unicode_literals
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import unicodedata
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import string
from urllib.parse import urlparse


In [3]:
#Clean de jojo
def clean(words):
    w = words.split(" ")
    a = list()
    english_stopwords = set(
        [stopword for stopword in stopwords.words('english')])
    punctuation = set(string.punctuation)
    punctuation.update(["``", "`", "...", ""])
    stemmer = SnowballStemmer('english')
    for word in w:
        if word.startswith("http"):
            a.append('{uri.netloc}'.format(uri=urlparse(word))) 
        else: 
            word = word.lower()
            #print(word)
            for arr in re.split("[, \-\()/’~'*0123456789!?:.;\"]+", word):
                if arr not in english_stopwords and arr not in punctuation and len(arr)>1:
                    a.append(stemmer.stem(arr))
    #print(a)
    return ' '.join(a)


In [95]:
data_myers = pd.read_csv('data/mbti.csv', sep=',')

In [96]:
data_myers.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [101]:
data_myers.type.unique()

array(['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'], dtype=object)

In [97]:
X_df = data_myers[:400].copy()

In [98]:
X_df.shape

(400, 2)

In [99]:
X_df.loc[:,'posts'] = X_df['posts'].apply(lambda x: x.split("|||")) #ici
X_df.loc[:,'posts']  = X_df['posts'].apply(lambda click: [(word.replace('"'," ")).replace("'","") for word in click])
X_df.loc[:,'posts']  = X_df['posts'].apply(lambda click: ['{uri.netloc}'.format(uri=urlparse(word)) if word.startswith("http") else word for word in click])
X_df.loc[:,'posts']  = X_df['posts'].apply(lambda click: [clean(words) for words in click])
X_df.loc[:,'posts']  = X_df['posts'].apply(lambda click: ' '.join(click))

In [100]:
X_df.head()

,type,posts
0,INFJ,www youtub com media tumblr com enfp intj mome...
1,ENTP,im find lack post alarm sex bore posit often e...
2,INTP,good one _____ www.youtube.com cours say know ...
3,INTJ,dear intp enjoy convers day esoter gab natur u...
4,ENTJ,your fire that anoth silli misconcept approach...


In [57]:
data_myers.shape

(8675, 2)

In [58]:
#Stratified resampling
X = data_myers.posts
y = data_myers.type
sss = StratifiedShuffleSplit(test_size=0.3)
sss.get_n_splits(X,y)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [59]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

train_to_csv = pd.concat(objs = [X_train,y_train] ,axis=1)
test_to_csv = pd.concat(objs = [X_test,y_test] ,axis=1)
train_to_csv.head()

,posts,type
0,'As the MOST introverted Extroverts I am begin...,ENTP
1,"'damn! you got me!|||I completely, totally and...",INTP
2,"'Dear (ISTP) Yeti, We're into our 5th year as ...",INFJ
3,'While depression is only a small part of my i...,INFP
4,'635770 One of the best films ever....|||I'd ...,INFP


In [51]:
#test_to_csv['posts'] = test_to_csv['posts'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))

In [60]:
test_to_csv.to_csv(path_or_buf = r'C:\Users\Dinojan\Desktop\ING5\DataCamp\DataCamp week\PROJET_RAMP-20171220-092500\test.csv', header=True, index=False)

In [61]:
train_to_csv.to_csv(path_or_buf = r'C:\Users\Dinojan\Desktop\ING5\DataCamp\DataCamp week\PROJET_RAMP-20171220-092500\train.csv', header=True, index=False)